In [1]:
import pandas as pd
import arff as liacarff

# 加载 ARFF 文件
with open('PROMISE_DOSSPRE_OLD_3.arff', 'r') as f:
    data_dict = liacarff.load(f)

# 提取数据
data = data_dict['data']

# 转换为 DataFrame
df = pd.DataFrame(data, columns=[attr[0] for attr in data_dict['attributes']])
df['ProjectID'] = df['ProjectID'].astype(int)  # 将 ProjectID 列转换为整数类型

# 显示 DataFrame
print(df)

      ProjectID                                    RequirementText _class_
0             1  The system shall refresh the display every 60 ...      PE
1             1  The application shall match the color of the s...      LF
2             1  If projected  the data must be readable.  On a...      US
3             1  The product shall be available during normal b...       A
4             1  If projected  the data must be understandable....      US
...         ...                                                ...     ...
2052          9  The database may trade off fidelity through ca...      FT
2053          9  The API shall have master topology replicating...      FT
2054          9  The system must parse, filter, transform and s...      FT
2055          9  The application shall employ real-user monitor...      FT
2056          9  The software should apply graceful degradation...      FT

[2057 rows x 3 columns]


In [2]:
# DATA PREPROCESSING
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string

# 下载NLTK的停用词和词性标注器数据
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# lower
# df['RequirementText'] = df['RequirementText'].str.lower()

# # Remove punctuation, leading and trailing spaces
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(f"[{re.escape(string.punctuation)}]", "", x.strip()))

# # 分词
df['RequirementText'] = df['RequirementText'].apply(word_tokenize)

# # stopwords
# stop_words = set(stopwords.words('english'))
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [word for word in x if word not in stop_words])

# # stemming
# stemmer = PorterStemmer()
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [stemmer.stem(word) for word in x])

# # lemmatization
# lemmatizer = WordNetLemmatizer()
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# # 合并词语为字符串
df['RequirementText'] = df['RequirementText'].apply(lambda x: ' '.join(x))

# Replace multiple spaces with a single space
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(' +', ' ', x))

# 移除整个列中每个字符串首尾的全部空格
df['RequirementText'] = df['RequirementText'].str.strip()

# 查看处理后的数据集
print(df.head())

[nltk_data] Downloading package stopwords to /home/li/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/li/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/li/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


   ProjectID                                    RequirementText _class_
0          1  The system shall refresh the display every 60 ...      PE
1          1  The application shall match the color of the s...      LF
2          1  If projected the data must be readable On a 10...      US
3          1  The product shall be available during normal b...       A
4          1  If projected the data must be understandable O...      US


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [4]:
from sklearn.model_selection import train_test_split
y = df['_class_']
X = df['RequirementText']

# 假设 X 是特征，y 是目标变量（类别标签）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = CountVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df)

features_train = tfidf_vectorizer.fit_transform(X_train).toarray()
labels_train = y_train
features_test = tfidf_vectorizer.transform(X_test).toarray()
labels_test = y_test

In [6]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

feature_names = tfidf_vectorizer.get_feature_names_out()
input_dimension = len(feature_names)

label_encoder = LabelEncoder()
labels_train_encoded = label_encoder.fit_transform(labels_train)
labels_test_encoded = label_encoder.transform(labels_test)

labels_train_one_hot = to_categorical(labels_train_encoded)
labels_test_one_hot = to_categorical(labels_test_encoded)

model = Sequential()
model.add(Dense(24, input_dim=input_dimension, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='softmax'))

# 创建 EarlyStopping 回调
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型，并使用 EarlyStopping 回调
history = model.fit(features_train, labels_train_one_hot, epochs=100, batch_size=16, 
                    validation_data=(features_test, labels_test_one_hot),
                    callbacks=[early_stopping])

accuracy = model.evaluate(features_test, labels_test_one_hot)[1]
print("Test Accuracy:", accuracy)

y_pred_prob = model.predict(features_test)
y_pred = np.argmax(y_pred_prob, axis=1)

report = classification_report(np.argmax(labels_test_one_hot, axis=1), y_pred)
print(report)

2023-12-08 13:10:54.371526: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-08 13:10:54.603345: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 13:10:54.603513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 13:10:54.623254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 13:10:54.698140: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/100
116/116 [==============================] - 1s 3ms/step - loss: 2.3282 - accuracy: 0.2080 - val_loss: 2.1814 - val_accuracy: 0.2864
Epoch 2/100
116/116 [==============================] - 0s 2ms/step - loss: 2.0366 - accuracy: 0.3431 - val_loss: 1.9284 - val_accuracy: 0.4272
Epoch 3/100
116/116 [==============================] - 0s 2ms/step - loss: 1.7211 - accuracy: 0.4673 - val_loss: 1.6974 - val_accuracy: 0.5243
Epoch 4/100
116/116 [==============================] - 0s 2ms/step - loss: 1.4672 - accuracy: 0.5667 - val_loss: 1.5023 - val_accuracy: 0.5777
Epoch 5/100
116/116 [==============================] - 0s 2ms/step - loss: 1.2726 - accuracy: 0.6272 - val_loss: 1.3765 - val_accuracy: 0.5922
Epoch 6/100
116/116 [==============================] - 0s 2ms/step - loss: 1.1339 - accuracy: 0.6623 - val_loss: 1.3063 - val_accuracy: 0.5922
Epoch 7/100
116/116 [==============================] - 0s 1ms/step - loss: 1.0284 - accuracy: 0.6823 - val_loss: 1.2412 - val_accuracy: 0.6262

In [7]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# 定义模型构建函数
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=8, max_value=512, step=8),
                    activation='relu', input_dim=input_dimension))
    model.add(Dense(units=hp.Int('units_2', min_value=8, max_value=512, step=8),
                    activation='relu'))
    model.add(Dense(12, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# 创建 EarlyStopping 回调
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 创建调优器
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # 试验的次数
    executions_per_trial=3,  # 每个试验运行的次数
    directory='my_tuning_dir',  # 存储结果的目录
    project_name='my_neural_network_tuning',
    overwrite=True
)

# 运行调优
tuner.search(features_train, labels_train_one_hot, epochs=100, batch_size=16, 
             validation_data=(features_test, labels_test_one_hot),
             callbacks=[early_stopping])

# 获取最佳模型
best_model = tuner.get_best_models(num_models=1)[0]

# 评估最佳模型
accuracy = best_model.evaluate(features_test, labels_test_one_hot)[1]
print("Test Accuracy (Best Model):", accuracy)

# 预测最佳模型的输出
y_pred_prob = best_model.predict(features_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# 输出分类报告
report = classification_report(np.argmax(labels_test_one_hot, axis=1), y_pred)
print(report)

Trial 5 Complete [00h 00m 07s]
val_accuracy: 0.6618122855822245

Best val_accuracy So Far: 0.7055016160011292
Total elapsed time: 00h 00m 35s
7/7 [==============================] - 0s 2ms/step - loss: 1.0897 - accuracy: 0.7136
Test Accuracy (Best Model): 0.7135922312736511
7/7 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.83      0.66      0.73        44
           2       1.00      0.71      0.83         7
           3       0.86      0.67      0.75         9
           4       0.58      0.88      0.70        16
           5       0.59      0.62      0.61        16
           6       0.75      0.69      0.72        13
           7       0.62      0.80      0.70        25
           8       0.56      0.62      0.59         8
           9       0.90      0.90      0.90        10
          10       0.71      0.71      0.71        24
          11       0.68   

In [8]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

features_train_resampled = tfidf_vectorizer.fit_transform(X_train_resampled).toarray()
labels_train_resampled = y_train_resampled
features_test_resampled = tfidf_vectorizer.transform(X_test_resampled).toarray()
labels_test_resampled = y_test_resampled

feature_names = tfidf_vectorizer.get_feature_names_out()
input_dimension = len(feature_names)

label_encoder = LabelEncoder()
labels_train_encoded_resampled = label_encoder.fit_transform(labels_train_resampled)
labels_test_encoded_resampled = label_encoder.transform(labels_test_resampled)

labels_train_one_hot_resampled = to_categorical(labels_train_encoded_resampled)
labels_test_one_hot_resampled = to_categorical(labels_test_encoded_resampled)

tuner.search(features_train_resampled, labels_train_one_hot_resampled, epochs=100, batch_size=16, 
             validation_data=(features_test_resampled, labels_test_one_hot),
             callbacks=[early_stopping])

# 获取最佳模型
best_model = tuner.get_best_models(num_models=1)[0]

# 评估最佳模型
accuracy = best_model.evaluate(features_test, labels_test_one_hot)[1]
print("Test Accuracy (Best Model):", accuracy)

# 预测最佳模型的输出
y_pred_prob = best_model.predict(features_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# 输出分类报告
report = classification_report(np.argmax(labels_test_one_hot, axis=1), y_pred)
print(report)

ValueError: could not convert string to float: 'The system shall maintain customer email information as a required part of customer profile'